# cours

## ré-importation (classes)

In [18]:
# modules importants pour le(s) tp(s) :

# visualisation des couleurs et autres... (partie graphique)
from dash import Dash, html # type: ignore
import dash_cytoscape as cyto # type: ignore

# partie système
from time import time, sleep
import os, sys, random as rd
from random import randint

# partie mathématique + structure
import numpy as np # type: ignore
from math import inf, sqrt, cos, sin, tan

# ON IMPORTE LES CLASSES DU TP PRECEDENT
class V:
    '''représente les informations des sommets (vertex)'''
    def __init__(self, name:str | int, weight=0):
        self.name = name
        self.weight = weight
class Vertex:
    """
    représente les sommets et leurs adjacences dans un graphe
    
    on crée la classe qui s'occupe de toutes les adjacences entre 
    les sommets (vertex/vertices) et leurs autres sommets reliés
    les arrêtes (edges) ne sont pas représentés car c'est la matrice d'adjacence qui s'occupe de ça
    qui est situé dans la classe Graph
    """
    id = 0 # identifiant du sommet 

    def __init__(self, name:str, neighbor:list[V]):
        self.name = name
        self.total_weight = sum([v.weight for v in neighbor]) # poids total du sommet
        self.neighbor = neighbor
        self.vertices_names = [v.name for v in self.neighbor]
        self.degree = len(neighbor)
        self.id = Vertex.id
        Vertex.id += 1

    def __str__(self):
        '''si on affiche Node (print(Node)) renverra ce qui suit'''
        res = f"{self.name} ["
        v_size = len(self.neighbor)
        for i, v in enumerate(self.neighbor):
            res += f"{v.name} w={v.weight}"
            if(i!=v_size-1):
                if v_size>1:
                    res+="," 
                res+=" "
        res+="]"
        return res
class Edge:
    '''représente une arrête entre deux sommets/vertex
    manière différente de représenter un graphe'''

    id = 0

    def __init__(self, vfrom:str | int, vto:str | int, weight: int | float):
        self.a = vfrom
        self.b = vto
        self.weight = weight
        self.id = Edge.id
        Edge.id += 1
    
    def __eq__(self, other:"Edge"): # equivalent to self == other
        '''permet d'utiliser l'objet dans un set (le rendre hashable)
        pour faire des comparaisons entre des ensembles (set) d'objet Edge'''
        # On considère que deux arêtes sont égales si leurs sommets et poids sont égaux
        return (self.a == other.a and self.b == other.b and self.weight == other.weight)
    
    def __ne__(self, edge:object): # equivalent de !=
        return not self.__eq__(edge)    

    def __hash__(self): # equivalent to set(self) == set(other)
        """permet d'utiliser l'objet dans un set (le rendre hashable)
        pour faire des comparaisons entre des ensembles (set) d'objet Edge"""
        # Calculer le hash en fonction des attributs de l'objet
        return hash((self.start, self.end, self.weight))

class Graph:
    """créée un graphe avec une liste de "Node" en paramètre et un titre
    @version 3.0.1"""

    def __init__(self, data:list[Vertex], title:str="default"): # OK
        '''@version 3.0.0'''
        self.title = title # titre du graphique si utilisé
        self.adj = data # liste d'adjacences
        self.edges = []
        self.verticesn = [] # liste des sommets (seulement les noms)
        self.weight = 0 # poids total du graphe    
        self.degree = 0 # pas encore calculé le degré du graphe
        self.is_complete = True
        for vx in self.adj:
            self.verticesn.append(vx.name)
            self.weight += vx.total_weight
            for v in vx.neighbor:
                self.edges.append(Edge(vx.name, v.name, v.weight))
                self.degree += 1
        self.create_matrix() # matrice des liens entre les noeuds (Edges) (matrice d'adjacences)
        self.port = 8054+randint(10,round(1e4))
        self.show_weights=True
        self.color_edges=[] # arretes a colorer (UGI)
        self.is_colored=True # autorise la coloration (UGI)

    def create_matrix(self): # OK
        """créer la matrice d'ajdacence des points\n
        (fonctionne avec les graphes orienté également)
        @version {3.0.1}
        prochaine version supprimer l'initialisation de la matrice a False
        et ajouter un par un les True et False (reduction de complexité)
        """
        # si on trouve que m[i][j] != m[j][i] 
        # ou que les poids ne sont pas les mêmes c'est oriente
        # attention dans les Vertex les V peuvent ne pas être dans l'ordre
        self.matrix = [[False for _ in self.adj] for _ in self.adj]
        self.is_oriented=False
        for lin, vis in enumerate(self.adj):
            for v in vis.neighbor:
                v_jx_idx = self.verticesn.index(v.name)
                self.matrix[lin][v_jx_idx]=True
                if not self.is_oriented:
                    if vis.name not in self.adj[v_jx_idx].vertices_names:
                        self.is_oriented=True
                    else:
                        vi_xj_idx = self.adj[v_jx_idx].vertices_names.index(vis.name)
                        vi_ji = self.adj[v_jx_idx].neighbor[vi_xj_idx]
                        if v.weight != vi_ji.weight :
                            self.is_oriented=True
        iter = range(len(self.matrix))
        # matrice d'un graphe complet avec la liste des sommets actuel
        complete = [[self.matrix[i][j] if i == j else True for i in iter] for j in iter]
        if(self.matrix == complete):
            self.is_complete = True

    def __str__(self): # OK
        """si jamais on print un graph (print(Graph)) c'est executé ici
        affichage au plus simple du graphe avec des caractères
        @version 3.0.0"""
        res = ""
        for i, vis in enumerate(self.adj):
            res += f"{i}\t | {vis.name} ["
            v_size = len(vis.neighbor)
            for j in range(v_size):
                res += f"{vis.neighbor[j].name} w={vis.neighbor[j].weight}"
                if(j!=v_size-1):
                    if v_size>1:
                        res += "," 
                    res+=" "
            res += "]\n"
        return res
    
    def _sort_edges(self, edges:list[Edge])->list[Edge]: # OK complexite≈O(n+log(5n))
        """fonction privée a ne pas utiliser (en dehors de la classe)
        algorithme reccursif pour les problèmes de pronfondeur et de performances
        @version 3.0.0"""
        if len(edges)<2 :
            return edges
        else:
            pivot = edges[len(edges)//2].weight
            l, m, r = [],[],[] # mineurs, égal, majeurs 
            for e in edges:
                if(e.weight < pivot): l.append(e)
                    # si reccursion sur len(m) ce n'est jamais < 2
                    # et donc (boucle infini) dans certains cas                
                elif(e.weight == pivot): m.append(e) 
                else: r.append(e)
            return self._sort_edges(l)+m+self._sort_edges(r)

    def sort_by_weight(self): # OK
        """trie le graphe par poids croissants(asc)
        on trie chaque arrêtes du graphe
        @version 3.0.0
        """
        self.edges = self._sort_edges(self.edges)

    def show_edges(self):
        '''affiche les edges sous forme de liste de string
        @version 3.0.0'''
        print([e.a+e.b+" w="+str(e.weight) for e in self.edges])

    def resume(self):
        '''crée un résumé du graphe
        @version 3.0.0'''
        print("quelques informations sur le graphe : \n")
        print(f"\tdegré: {self.degree}")
        print(f"\tpoids: {self.weight}")
        print(f"\tcomplet : {self.is_complete} (def: si tous les sommets sont reliés)")
        print(f"\tplanaire: voir si les arrêtes se croisent ou non")
        print(f"\t\tnecessite surement de déplacer\n\t\tles noeuds (sur la partie graphique)")

# code de génération de couleurs selon le 
# nombre de parts (nb de couleurs nécessaire)

def generate_colors(n:int)->list[str]:
    """Génère N couleurs différentes en format hexadécimal, réparties uniformément.
    Les couleurs changent à chaque exécution grâce à un mélange aléatoire.
    """
    # Diviser l'espace des couleurs de manière uniforme
    step = 256 // n
    # Générer une palette de couleurs de base
    tmp = range(n)
    base_colors = [(i * step, j * step, k * step) for i in tmp for j in tmp for k in tmp]
    rd.shuffle(base_colors) # mélange
    selected_colors = base_colors[:n] # les N premières couleurs
    # return conveted colors to hex
    return [f"#{r:02x}{g:02x}{b:02x}" for r, g, b in selected_colors]

def int_to_rgb(color_int_:int)->str:
    '''convertit une valeur int tres grande (255**3) en rgb'''
    if type(color_int_) != int:
        raise ValueError(f"(function) int_to_rgb(n=\"{color_int_}\"), n is not int")
    color_int = color_int_ % (256**3+1)
    r = (color_int >> 16) & 0xFF
    g = (color_int >> 8) & 0xFF
    b = color_int & 0xFF
    return f"#{r:02x}{g:02x}{b:02x}"

def generate_colors2(n:int)->list[str]:
    """Génère N couleurs réparties uniformément 
    avec un décalage cyclique global aléatoire format rgb hex (#000000)"""
    max_colors = 256 ** 3  
    step = max_colors // n
    rd_decal = rd.randint(0, max_colors - 1) 
    # Générer les N couleurs en considérant l'espace continu
    colors = [(i * step + rd_decal) % max_colors for i in range(n)]
    # Convertir les couleurs décalées en format RGB puis en hexadécimal

    return [int_to_rgb(color) for color in colors]

class Graph(Graph): 
    # on ajoute des méthodes a l'existant (extension)

    def __init__(self, data, title = "default"):
        super().__init__(data, title)
        self._render()

    def _render(self):
        '''initialize graphic things'''
        self.custom_style = {
            'width': '100%', 
            'height': '500px',
            "border": "3px white solid",
            "border-radius":"5px",
            "background-color":"#666666",
            "title" : {"background-color":"white"}
        }        
        self.unique = f' {time()%1e4:.5}'
        allow_arrows = "linear" # ce style n'autorise pas les flèches
        if self.is_oriented:
            allow_arrows = "bezier" # ce style oui
        self.my_styles_sheet = [{
                'selector': 'node',
                'style': {
                    'background-color': '#222222', 
                    'color': 'white',
                    'label': 'data(label)',
                    'font-size': '16px',
                    'text-valign': 'center', 
                    'text-halign': 'center' 
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'width': 2,
                    'target-arrow-shape': "vee",
                    "target-arrow-color": "#4a7cf2",
                    'arrow-scale': 2,
                    'curve-style': allow_arrows
                }
            }
        ]

    def render(self, layout_name="breadthfirst"): # rendu UGI
        """effectue le rendu du graphe visuellement
        @version 3.1.4"""
        if self.show_weights:
            self.my_styles_sheet.append({
                'selector': 'edge',
                'style': {
                    'label': 'data(weight)',
                    "color": 'white'
                }
            })   
        elems = [] # éléments à afficher (formattés)
        for node in self.adj:
            elems.append({'data': {"id":node.name, "label":node.name}})
        # add edges
        for edge in self.edges:
            elems.append({
                'data': {
                    'source': edge.a, 
                    'target': edge.b, 
                    'weight': edge.weight
                }
            })
        app = Dash(self.title+self.unique)            
        app.layout = html.Div([
            cyto.Cytoscape(
                id='cytoscape'+self.unique,
                elements=elems,
                layout={'name': layout_name},
                style=self.custom_style,
                stylesheet=self.my_styles_sheet
            )
        ])
        print('\nrendu graphique : ')
        print(f"\tégalement ouvert sur la page web : \"localhost:{self.port}\"")
        print(f"\topened too at the web page : \"localhost:{self.port}\"")
        app.run_server(debug=True,port=self.port)

    def _color_grg(self, nb_colors:int):
        '''permet de colorer le graphe affiché avec cytoscape'''
        color = generate_colors(nb_colors)
        print(f"nbcol:{nb_colors}result={color}")
        for k in self.res["data"]:
            v = self.res["data"][k]
            self.my_styles_sheet.append(
                { # la règle la plus importante a la fin (override des précédentes)
                    'selector': f'node[id="{k}"]',
                    'style': {
                        'background-color': f'{color[v-1]}',
                    }
                }                    
            )

    def coloration(self)->dict[str:int, str:dict[str:int]]:
        """
        Réalise une coloration des sommets d'un graphe donné.
        renvoie un dict: Un dictionnaire associant chaque sommet à une couleur.
        """
        sommets_tries = sorted(self.adj, key=lambda v: len(v.neighbor), reverse=True)
        # Initialiser les couleurs
        coloration = {vertex.name: None for vertex in self.adj}
        couleur_max = 0
        # Attribuer les couleurs
        for vertex in sommets_tries:
            # Collecter les couleurs déjà utilisées par les voisins
            couleurs_voisines = [
                coloration[neighbor.name]
                for neighbor in vertex.neighbor
                if coloration[neighbor.name] is not None
            ]
            # Trouver la première couleur non utilisée
            couleur = 1
            while couleur in couleurs_voisines:
                couleur += 1
            coloration[vertex.name] = couleur
            couleur_max = max(couleur_max, couleur)
            coloration[vertex.name] = couleur
            couleur_max = max(couleur_max, couleur)
        # pour voir le résultat de l'extérieur            
        self.res=dict()
        self.res["Nombre de couleurs utilisées"]=couleur_max
        self.res["data"] = coloration
        self._color_grg(couleur_max) # rendu graphique

        

## formulas/formules

$$
init \left\{
\begin{array}{l}
P_1(1,1) = 0 \\
P_1(1,2) = P_{12} = 4 \\
P_1(1,3) = P_{13} = 6 \\
P_1(1,4) = P_1(1,5) = P_1(1,6) = \inf
\end{array}
\right.
$$

$$
P_2(1,2)=min(P_1(1,2), \underset{=P_1(1,1)+P_{12}}{\underset{j\in{\{1,3\}}}{\underbrace{min}}}(P_1(1,3)+P_{32})) = min(4,6+9) = 4
$$

$$
P_2(1,3)=min(P_1(1,3), \underset{j\in{\{1,4\}}}{\underbrace{min}}(P_1(1,4)+P_{43})) = min(6,\inf+1) = 6
$$

$A =$ Ensemble des arcs

$P_k(1,i)$=longueur minimale du chemin depuis le sommet 1 jusq'au sommet i en passant par au plus k arcs.

$$
\begin{array}{l}
init\ k=1 \\
P_1(1,i)= \end{array}\left\{
\begin{array}{l}
P_{1i} (poids de l'arc (1,i), \\
si \in Predecesseur(i) (ou (1,i) \in A)) \\
\end{array}
\right.
$$

$$
k = 2 \\
P_2(1,2) = min(\underset{=4}{P_1(1,2)})
$$


## Algorithme de Bellman ($O(n^2)$)

<!-- a -->
### etape 1
$P_1(1,1)=0$<br>
$P_1(1,2)=P_{12}=4$<br>
$P_1(1,3)=P_{13}=6$<br>
$P_1(1,4)=P_1(1,5)=P_1{(1,6)}=\inf$<br><br>
### etape 2
$h=2$<br>
$P_2(1,1) = 0$<br>
$P_2(1,2)=min(
    \underset{
        =4
    }{
        P_1(1,3)
    }, \underset{
        pred(2)=\{1,3\}
    }{
        \underbrace{min}
    }(
    \underset{
        =4
    }{
        \underbrace{
            \underset{=0}{\underbrace{P_1(1,1)}}
            +\underset{=4}{\underbrace{P_{12}}}
        }
    })) = min(6,\inf+1
) = 6 $<br>
### etape 3
$P_3(1,1)=0$<br>
$P_3(1,2)=min(
    4, \underset{
            pred(2)=\{1,3\}
        }{
            \underbrace{min}
        } (\ \ \ \ \ \ \ \ \ \ \ \ \ \ )
) = 4 $<br>
$P_3(1,3)=min(
    6, \underset{
        pred(3)=\{1,4\}
    }{
        \underbrace{min}
    } (6,P_2(1,4)+P_{43})
) = min(6, 6,7+1) = 6$<br><br>
$P_3(1,4)=min(7,\underset{pred(4)=\{2\}}{\underbrace{min}}(P_2(1,2)+P_{14}))$<br>
### etape 4

a écrire

## Algorithme de Floyd-Warshall ($O(|n|^3)$)


algorithme : <br><br>
$pour\ k=1,...,n$<br>
$\ \ \ \ pour\ i=1,...,n$<br>
$\ \ \ \ \ \ \ \ pour\ j=1,...,n$<br>
$\ \ \ \ \ \ \ \ \ \ \ \ C_k(i,j)=min(C_{k-1}(i,j),C_{k-1}(i,j)+C_{k-1}(k_{ij}))$<br>
$\ \ \ \ \ \ \ \ endp$<br>
$\ \ \ \ endp$<br>
$endp$<br>


$
L = 
    \begin{matrix}
    1 \\ 2 \\ 3 \\ 4 \\ 5 \\ 6
    \end{matrix}  
    \overset{ 1 \ \ \ 2 \ \ \ 3 \ \ \ 4 \ \ \ 5 \ \ \ 6}{  
    \begin{pmatrix}
    0 & 4 & 6 & \infty & \infty & \infty \\
    \infty & 0 & \infty & 3 & \infty & \infty \\
    \infty & 9 & 0 & \infty & 4 & \infty \\
    \infty & \infty & 1 & 0 & 1 & 4 \\
    \infty & \infty & \infty & \infty & 0 & 2 \\
    \infty & \infty & \infty & \infty & \infty & 0
    \end{pmatrix}
    }
$


solution : 
```
M =[
    [0,4,6,inf,inf,inf]
    [0,4,6,7,10,inf],
    [0,4,6,7,8,11],
    [0,4,6,7,8,10],
    [0,4,6,7,8,10] <-- solution
]
```

exemple matriciel

```
M = [
    [0,2,5,6,inf,inf]
    [inf,0,1,inf,inf,7]
    [inf,inf,0,inf,1,inf]
    [inf,inf,inf,0,inf,6]
    [inf,inf,inf,inf,0,1]
    [inf,n,n,n,inf,0]
]
```

minimum du calcul de la somme d'une ligne + colonne

$
M^0 =
    \begin{matrix}
    1 \\ 2 \\ 3 \\ 4 \\ 5 \\ 6
    \end{matrix}  
    \begin{pmatrix}
    0 & 2 & 5 & 6 & \infty & \infty \\
    \infty & 0 & 1 & \infty & \infty & 7 \\
    \infty & \infty & 0 & \infty & 1 & \infty \\
    \infty & \infty & \infty & 0 & \infty & 6 \\
    \infty & \infty & \infty & \infty & 0 & 1 \\
    \infty & \infty & \infty & \infty & \infty & 0
    \end{pmatrix}, \
M^1=
    \begin{matrix}
    1 \\ 2 \\ 3 \\ 4 \\ 5 \\ 6
    \end{matrix}  
    \begin{pmatrix}
    0 & 4 & 6 & \infty & \infty & \infty \\
    \infty & 0 & \infty & 3 & \infty & \infty \\
    \infty & 9 & 0 & \infty & 4 & \infty \\
    \infty & \infty & 1 & 0 & 1 & 4 \\
    \infty & \infty & \infty & \infty & 0 & 2 \\
    \infty & \infty & \infty & \infty & \infty & 0
    \end{pmatrix}, \
M^2=
    \begin{matrix}
    1 \\ 2 \\ 3 \\ 4 \\ 5 \\ 6
    \end{matrix}  
    \begin{pmatrix}
    0 & 2 & 3 & 6 & 4 & 5 \\
    \infty & 0 & 1 & \infty  & 2 & 3 \\
    \infty & \infty & 0 & \infty & 1 & 2 \\
    \infty & \infty  & \infty  & 0 & \infty  & 6 \\
    \infty & \infty & \infty  & \infty & 0 & 1 \\
    \infty & \infty & \infty & \infty & \infty & 0
    \end{pmatrix}
$

$
M^3=
    \begin{matrix}
    1 \\ 2 \\ 3 \\ 4 \\ 5 \\ 6
    \end{matrix}  
    \begin{pmatrix}
    0 & 2 & 3 & 6 & 4 & 5 \\
    \infty & 0 & 1 & \infty  & 2 & 3 \\
    \infty & \infty & 0 & \infty & 1 & 2 \\
    \infty & \infty  & \infty  & 0 & \infty  & 6 \\
    \infty & \infty & \infty  & \infty & 0 & 1 \\
    \infty & \infty & \infty & \infty & \infty & 0
    \end{pmatrix}
$

dans $M^3$, pour un élément de la colonne I a un plus courte chemin de valeur indiquée dans la matrice jusqu'a un élément de la ligne J noté dans la matrice on a $M_{IJ} =$ plus court chemin

### implémentation de Floyd-Warshall

In [16]:
class Graph(Graph):

    def __init__(self, data, title="default"):
        super().__init__(data, title)

    def floyd_warshall(self):
        pass

## Algorithme de Dijkstra

- a chaque étapes, on détermine la distance entre le sommet 1 et un nouveau sommet, dans l'ordre croissan des distances.
- Les longueurs/poids +
- donne le chemin de longueur min entre deux sommets

```
init
    !S={2,3,...n}
    P*(1)=0 
    P(i) = 1i si i pred (1), inf sinon

// complemtaire S = !S (avec une barre au dessus)
while complementaire S not None:
    select j tel que p(j) = min(p(k)) in !S
    !S=!S/{j}
    p*(j)=P(j)
    for tout i in !S cross successeur(j):
        p(i) = min(p(i), p(j)+l[ij])
```


exemple : 

graphe : <br>

1: 1, 3<br>
2: 4, 6<br>
3: 2,5,6<br>
4:  <br>
5: 4,2<br>
6: 5<br>

$\bar{S}$ = {1}<br>
$\bar{S}=${1,2,3,4,5,6}<br>
$P^*(1)$=0<br>

P(2)=7,P(3)=1,P(4)=P(5)=P(6)=$\infty$

- j tel que P(j)=min(p(k))=min(7,1,$\infty$,$\infty$,$\infty$)=1 $\longmapsto$ j=3
- $\bar{S}=\bar{S}$-{3}={2,4,5,6}
- $P^*(3)$=1
- $
\begin{array}{l}
\bar{S}  \cap Successseur(3)=\{2,5,6\} \end{array}\left\{
\begin{array}{l}
P(2)=min(\overset{=7}{P(2)}, \overset{=1}{P(3)}+\overset{=5}{P_{32}}) = 6 \\
P(5)=min(\overset{=\infty}{P(5)},P(3)+\overset{=2}{P_{35}}) = 3 \\
P(6)=min(\overset{=\infty}{P(6)},P(3)+\overset{=7}{P_{36}})=8 \\
\end{array}
\right.
$

<hr>

- j tel que p(j) = min(P(k)) $\longmapsto$ j=5
- $\bar{S}$ = {2,4,6}, S={1,3,5}, P*(5)=3
- $\bar{S}\ \cap $ Sucesseur(5) = {4,2}
<hr>

- $\bar{S} \neq \varnothing: j \longmapsto$ 2, $\bar{4,6}$ S={1,2,3,5}
- $\bar{S}\ \cap$ Sucesseur(2) = {4,6}
<hr>

- $\bar{S}\neq \varnothing : j \longmapsto \bar{S} $ ={4}

<hr>

- $\bar{S}\neq \varnothing$ 

## Test d'algo

In [10]:
# algo de Dijkstra

class Graph(Graph):

    def __init__(self, data, title="default"):
        super().__init__(data, title)

    def dijkstra(self):
        nS = set(self.adj)
        vide = set()
        P=0
        while nS != vide :
            j = min([e.weight for e in nS])


"""        
    !S={2,3,...n}
    P*(1)=0 
    P(i) = 1i si i pred (1), inf sinon

// complemtaire S = !S (avec une barre au dessus)
while complementaire S not None:
    select j tel que p(j) = min(p(k)) in !S
    !S=!S/{j}
    p*(j)=P(j)
    for tout i in !S cross successeur(j):
        p(i) = min(p(i), p(j)+l[ij])"""

'        \n    !S={2,3,...n}\n    P*(1)=0 \n    P(i) = 1i si i pred (1), inf sinon\n\n// complemtaire S = !S (avec une barre au dessus)\nwhile complementaire S not None:\n    select j tel que p(j) = min(p(k)) in !S\n    !S=!S/{j}\n    p*(j)=P(j)\n    for tout i in !S cross successeur(j):\n        p(i) = min(p(i), p(j)+l[ij])'

In [11]:
# test
data = [
    Vertex("A", [V("B")]),
    Vertex("B", [V("C")]),
    Vertex("C", [V("G"),V("D")]),
    Vertex("D", [V("E")]),
    Vertex("E", [V("F"),V("A")]),
    Vertex("F", [V("G")]),
    Vertex("G", [V("B")])
]
graphe = Graph(data, "test")
#graphe.show_weights = False
graphe.render()


rendu graphique : 
	également ouvert sur la page web : "localhost:16213"
	opened too at the web page : "localhost:16213"


## récapitulatif

|          | Résultat                                                                            | poids |
| -------- | ----------------------------------------------------------------------------------- | ----- |
| Bellman  | le chemin optimal <br>du sommet 1 au<br>sommet n pour<br>n $\in$ X (tout sommet)    | +/-   |
| Floyd    | Le chemin optimal entre<br>tout pair de sommets.<br>(entre tous les sommets)        | +/-   |
| Dijkstra | Le chemin optimal du<br>sommmet 1 au sommet n,<br>pour tout n $\in$ X (tout sommet) | +     |

## exercices

In [ ]:
# données
data = [
    Vertex("A", [V("B",14),V("H",11)]),
    Vertex("B", [V("C",6),V("F",5),V("E",2)]),
    Vertex("C", [V("G",3),V("J",5)]),
    Vertex("D", [V("F",10),V("E",13),V("A",1)]),
    Vertex("E", [V("F",1),V("B",2)]),
    Vertex("F", [V("E",1),V("G",14),V("I",10)]),
    Vertex("G", []),
    Vertex("H", [V("A",11)]),
    Vertex("I", [V("H",12)],V("J",7)),
    Vertex("J", [V("C",5)])
]
graph = Graph(data)
graph.render()
print("trouver le plus court chemin depuis D (\"1\") à tout les autres sommets")


rendu graphique : 
	également ouvert sur la page web : "localhost:10104"
	opened too at the web page : "localhost:10104"


trouver le plus court chemin depuis D ("1") à tout les autres sommets


[[False, True, False, False, False, False, False, True, False, False],
 [False, False, True, False, True, True, False, False, False, False],
 [False, False, False, False, False, False, True, False, False, True],
 [True, False, False, False, True, True, False, False, False, False],
 [False, True, False, False, False, True, False, False, False, False],
 [False, False, False, False, True, False, True, False, True, False],
 [False, False, False, False, False, False, False, False, False, False],
 [True, False, False, False, False, False, False, False, False, False],
 [False, False, False, False, False, False, False, True, False, False],
 [False, False, True, False, False, False, False, False, False, False]]

## Longueur du chemin optimale de A $\longmapsto$ E

In [21]:
# données
data = [
    Vertex("A", [V("B",3),V("C",2),V("E",4)]),
    Vertex("B", [V("D",4),V("C",-2),V("E",-1)]),
    Vertex("C", [V("D",3),V("E",2)]),
    Vertex("D", [V("E",2)]),
    Vertex("E", [])
]
print("on utilise Bellman")
graph = Graph(data)
print(graph)
matr_result_expect = [
    [0,3,2,inf,4],
    [0,3,1,5,2],
    [0,3,1,4,2],
    [0,3,1,4,2]
]

# réponse : de A à E c'est 2 (A B E)


on utilise Bellman
0	 | A [B w=3, C w=2, E w=4]
1	 | B [D w=4, C w=-2, E w=-1]
2	 | C [D w=3, E w=2]
3	 | D [E w=2]
4	 | E []

